In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
edge_data = pd.read_csv('Edge_Data.csv', index_col = [0])
node_data = pd.read_csv('Node_Data.csv', index_col = [0])

In [3]:
node_data['id'] = node_data['id'].apply(int)
node_data['latitude'] = node_data['latitude'].apply(float)
node_data['longitude'] = node_data['longitude'].apply(float)

edge_data['start_id'] = edge_data['start_id'].apply(int)
edge_data['end_id'] = edge_data['end_id'].apply(int)

In [4]:
node_set = set(node_data['id'].tolist()) # set of all non-redundant nodes
node_id_to_index = {} # maps node id to index of csv file
node_adj = {} # all edges incident to a node

for index, row in node_data.iterrows():
    node_id_to_index[int(row['id'])] = index
    
for index, row in node_data.iterrows():
    string_of_edges = row['adjacencies']
    list_of_edges = string_of_edges.split('-')
    list_of_edges_int = [int(b) for b in list_of_edges]
    node_adj[int(row['id'])] = set(list_of_edges_int)

display(node_data)
display(edge_data)

,id,latitude,longitude,adjacencies
3,33947072,37.863725,-122.244567,3-4
4,33947074,37.863798,-122.244472,86-5-6
10,33947087,37.863233,-122.242479,8-153-7
13,33947096,37.863602,-122.242829,10-11
16,33947107,37.864293,-122.243366,19238-12-12550
...,...,...,...,...
44266,8922083552,37.871697,-122.254621,20180-20181-20182
44267,8922083553,37.872017,-122.254466,7840-20183-7839
44273,8922083559,37.871338,-122.254412,20184-7836-7837
44274,8922083560,37.871710,-122.254436,20184-20185-20183


,name,start_id,end_id,highway,crime_count,light_count,component,start_lat,start_lon,end_lat,end_lon
0,Stonewall-Panoramic Trail,35718720,2790624066,track,0,0,0,37.862638,-122.244025,37.862768,-122.244107
1,Stonewall-Panoramic Trail,2790624066,2535392487,track,0,0,0,37.862768,-122.244107,37.862927,-122.244216
2,Stonewall-Panoramic Trail,2535392487,2790624087,track,0,0,0,37.862927,-122.244216,37.863661,-122.244528
3,Stonewall-Panoramic Trail,2790624087,33947072,track,0,0,0,37.863661,-122.244528,37.863725,-122.244567
4,Stonewall-Panoramic Trail,33947072,2532688215,track,0,0,0,37.863725,-122.244567,37.863761,-122.244562
...,...,...,...,...,...,...,...,...,...,...,...
20182,NaN,8922083552,6269419172,footway,0,0,0,37.871697,-122.254621,37.871706,-122.254499
20183,NaN,8922083553,8922083560,footway,0,0,0,37.872017,-122.254466,37.871710,-122.254436
20184,NaN,8922083560,8922083559,footway,0,0,0,37.871710,-122.254436,37.871338,-122.254412
20185,NaN,6269419172,8922083560,footway,0,0,0,37.871706,-122.254499,37.871710,-122.254436


In [5]:
edge_length = len(edge_data)
sw_lat, sw_lon, ne_lat, ne_lon = 37.84369, -122.33657, 37.91079, -122.22803
avg_lat = (sw_lat + ne_lat) / 2
cosine = math.cos(avg_lat * math.pi / 180)
degree = 69.1694444
feet_in_miles = 5280
edge_data['distance'] = [None for i in range(len(edge_data))]

def distance(start_lat, start_lon, end_lat, end_lon):
    lat_diff = end_lat - start_lat
    lon_diff = end_lon - start_lon
    lon_diff *= cosine
    lat_diff *= degree * feet_in_miles
    lon_diff *= degree * feet_in_miles
    return math.sqrt(lat_diff * lat_diff + lon_diff * lon_diff)

for i in range(edge_length):
    start_id = node_id_to_index[edge_data.at[i, 'start_id']]
    end_id = node_id_to_index[edge_data.at[i, 'end_id']]
    start_lat = node_data.at[start_id, 'latitude']
    start_lon = node_data.at[start_id, 'longitude']
    end_lat = node_data.at[end_id, 'latitude']
    end_lon = node_data.at[end_id, 'longitude']
    edge_data.at[i, 'distance'] = distance(start_lat, start_lon, end_lat, end_lon)
    
display(edge_data)

,name,start_id,end_id,highway,crime_count,light_count,component,start_lat,start_lon,end_lat,end_lon,distance
0,Stonewall-Panoramic Trail,35718720,2790624066,track,0,0,0,37.862638,-122.244025,37.862768,-122.244107,53.109536
1,Stonewall-Panoramic Trail,2790624066,2535392487,track,0,0,0,37.862768,-122.244107,37.862927,-122.244216,66.085009
2,Stonewall-Panoramic Trail,2535392487,2790624087,track,0,0,0,37.862927,-122.244216,37.863661,-122.244528,282.72429
3,Stonewall-Panoramic Trail,2790624087,33947072,track,0,0,0,37.863661,-122.244528,37.863725,-122.244567,25.694295
4,Stonewall-Panoramic Trail,33947072,2532688215,track,0,0,0,37.863725,-122.244567,37.863761,-122.244562,13.444345
...,...,...,...,...,...,...,...,...,...,...,...,...
20182,NaN,8922083552,6269419172,footway,0,0,0,37.871697,-122.254621,37.871706,-122.254499,35.111718
20183,NaN,8922083553,8922083560,footway,0,0,0,37.872017,-122.254466,37.871710,-122.254436,112.515781
20184,NaN,8922083560,8922083559,footway,0,0,0,37.871710,-122.254436,37.871338,-122.254412,135.855005
20185,NaN,6269419172,8922083560,footway,0,0,0,37.871706,-122.254499,37.871710,-122.254436,18.240224


In [6]:
edge_data.to_csv('Edge_Data.csv')